# Requisitos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import SGD

# Cargada de datos

In [ ]:
!gdown 1H4kSVl34uUNYAoK5irjxsS0RXAe_PP5_
!gdown 1Z_OUXBsfCRm2tDuMC2LL1B_1DmpeN9P1

In [ ]:
X_raw = np.load('X.npy')
X_raw = X_raw/255.
y = np.load('y.npy')
X = []
size = (28,28)
for x in X_raw:
    X.append(resize(x, size))
X = np.array(X)

In [ ]:
plt.figure(figsize=(20,10))
for i in range(25):
    plt.subplot(5,5, i+1)
    idx = np.random.choice(X.shape[0], 1)[0]
    plt.title('id:{} val:{}'.format(idx, y[idx] ))
    plt.imshow(-X[idx], cmap='gray')
    plt.axis('off')
plt.show()

# Entrenamiento del modelo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
# Ajustamos las dimensiones de las imágenes
if X_train.ndim == 3:
    X_train = X_train[...,None]
    X_test = X_test[..., None]
    print(X_train.shape, X_test.shape)

In [ ]:
bs = 16
lr = 0.0001
class_mapping = {"Platano": 0, "Pera": 1, "Manzana": 2}
y_train = np.array([class_mapping[label] for label in y_train])
y_test = np.array([class_mapping[label] for label in y_test])
y_train = y_train.astype(int)
y_test = y_test.astype(int)

model = Sequential(
    [
        Conv2D(32, 3, activation="relu", input_shape=(*size, 1)),
        MaxPool2D(),
        Conv2D(64, 3, activation="relu", padding="same"),
        MaxPool2D(),
        Conv2D(128, 3, activation="relu", padding="same"),
        MaxPool2D(),
        Flatten(),
        Dense(128, activation="relu"),  # modificar!
        Dense(3, activation="softmax"),
    ]
)  # no modificar

In [ ]:
optimizer1 = SGD(learning_rate=lr)
model.compile(optimizer= optimizer1, loss='sparse_categorical_crossentropy' , metrics=['accuracy'])
model.summary()

In [ ]:
log = model.fit(X_train, y_train, batch_size=bs, epochs=800, validation_data=(X_test, y_test))
model.save('modelo_entrenado.h5')

# Probando el modelo

In [ ]:
#Probando modelo
etiqueta_a_fruta = {0: 'Platano', 1: 'Pera', 2: 'Manzana'}

In [ ]:
idx = np.random.choice(X_test.shape[0], 1)[0]
im = X_test[idx]
label = y_test[idx]
nombre_fruta = etiqueta_a_fruta.get(label, 'Desconocida')

In [ ]:
plt.title('(test) id:{} val:{} ({})'.format(idx, label, nombre_fruta))
plt.axis('off')
plt.imshow(-im[:,:,0], cmap='gray')
plt.show()


In [ ]:
#Usando el modelo
salida = model.predict(im[None,:,:,:])[0]

In [ ]:
plt.figure()
plt.title("Predicción: {} ({}), Label: {}".format(salida.argmax(), nombre_fruta, label))
plt.ylabel("Probabilidad")
plt.xlabel("Clase")
plt.ylim([0, 1])
plt.bar(np.arange(len(salida)), salida)
plt.show()